In [ ]:
import os
import numpy as np
import pandas as pd
from flask import Flask, request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from collections.abc import Iterable
import pickle
from flask_cors import CORS
import math

app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return "Hello An!"


def weightedRating(x):
    v = x['Rating']
    R = x['AggregatedRating']
    return (v / (v + 51.0) * R) + (51.0 / (51.0 + v) * 4.56)

categoryBasedRecommender = pickle.load(open('categoryBasedRecommender.pkl', 'rb'))
metadataBasedRecommender = pickle.load(open('metadataBasedRecommender.pkl', 'rb'))

datasetDumpAlgolia = pickle.load(open('datasetDump.pkl', 'rb'))


# weighted popularity model
#qualifiedRecipes = pickle.load(open('qualifiedRecipes.pkl', 'rb'))


qualifiedRecipes = pickle.load(open('qualifiedRecipesCollaborative.pkl', 'rb'))
qualifiedRecipes['score'] = qualifiedRecipes.apply(weightedRating, axis=1)
df_rating = pd.DataFrame(qualifiedRecipes.groupby(by = ['RecipeId'])['score'].agg('mean')) 

filtered_ratings = pickle.load(open('filtered_ratings.pkl', 'rb'))
collaborativeMatrix = filtered_ratings.pivot(index = 'AuthorId', columns ='RecipeId', values = 'Rating').fillna(0)

recipe_metadata = qualifiedRecipes[['RecipeId','Name','RecipeCategory']]
recipe_data = filtered_ratings.merge(recipe_metadata,on='RecipeId')
knnMatrix = recipe_data.pivot(index = 'RecipeId', columns ='AuthorId', values = 'Rating').fillna(0)
    

@app.route('/popularity')
def weightedRatingPopularity():
    return qualifiedRecipes.to_json()


# category - got the data in weightedPopularRecipes and now creating the route function
@app.route('/category', methods=['POST'])
def getCategoryWiseRecommendations():
    data = request.get_json()
    RecipeCategory = data['keyword']
    percentile = 0.85
    categorySpecificRecipes = categoryBasedRecommender[categoryBasedRecommender['Category'] == RecipeCategory]
    RatingCounts = categorySpecificRecipes[categorySpecificRecipes['Rating'].notnull()]['Rating'].astype('int')
    RatingAverages = categorySpecificRecipes[categorySpecificRecipes['AggregatedRating'].notnull()][
        'AggregatedRating'].astype('int')
    C = RatingAverages.mean()
    m = RatingCounts.quantile(percentile)

    qualified = categorySpecificRecipes[
        (categorySpecificRecipes['Rating'] >= m) & (categorySpecificRecipes['Rating'].notnull()) & (
            categorySpecificRecipes['AggregatedRating'].notnull())][
        ['RecipeId', 'Name', 'Rating', 'AggregatedRating', 'ReviewCount']]
    qualified['Rating'] = qualified['Rating'].astype('int')
    qualified['AggregatedRating'] = qualified['AggregatedRating'].astype('int')

    qualified['wr'] = qualified.apply(
        lambda x: (x['Rating'] / (x['Rating'] + m) * x['AggregatedRating']) + (m / (m + x['Rating']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(12)

    return qualified.to_json()


# metadata
termFrequency2 = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.0, stop_words='english')
tfidMatrix2 = termFrequency2.fit_transform(metadataBasedRecommender['Metadata'])
cosineSimilarity2 = linear_kernel(tfidMatrix2, tfidMatrix2)

metadataBasedRecommender = metadataBasedRecommender.reset_index()
names2 = metadataBasedRecommender[['RecipeId', 'Name']]
indices2 = pd.Series(metadataBasedRecommender.index, index=metadataBasedRecommender['Name'])


@app.route('/metadata', methods=['POST'])
def getMetadataBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]
    if isinstance(idx, Iterable):
        for i in idx:
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:13]
            break
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:13]
    recipeIndices = [i[0] for i in similarityScores]
    return names2.iloc[recipeIndices].to_json()


# content with ratings considered
contentBasedRecommender = metadataBasedRecommender


@app.route('/content', methods=['POST'])
def getContentBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]

    if isinstance(idx, Iterable):
        recipeDump = pd.DataFrame()
        for i in idx:
            cnt = 0
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:26]
            recipeIndices = [i[0] for i in similarityScores]

            contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
                ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
            ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
                'Rating'].astype('int')
            ratingAverages = \
            contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
                'AggregatedRating'].astype('int')
            C = ratingAverages.mean()
            m = ratingCounts.quantile(0.60)
            qualifiedRecipes2 = contentBasedRecommendedRecipes[(contentBasedRecommendedRecipes['Rating'] >= m) & (
                contentBasedRecommendedRecipes['Rating'].notnull()) & (contentBasedRecommendedRecipes[
                                                                           'AggregatedRating'].notnull())]
            qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
            qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
            qualifiedRecipes2['wr'] = qualifiedRecipes2.apply(weightedRating, axis=1)
            qualifiedRecipes2 = qualifiedRecipes2.sort_values('wr', ascending=False).head(12)
            if cnt == 0:
                recipeDump = qualifiedRecipes2
            else:
                recipeDump.append(qualifiedRecipes2, ignore_index=True)
            cnt += 1
        recipeDump = recipeDump.sort_values('wr', ascending=False).head(12)
        return recipeDump.to_json()

    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:26]
        recipeIndices = [i[0] for i in similarityScores]

        contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
            ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
        ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
            'Rating'].astype('int')
        ratingAverages = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
            'AggregatedRating'].astype('int')
        C = ratingAverages.mean()
        m = ratingCounts.quantile(0.60)
        qualifiedRecipes2 = contentBasedRecommendedRecipes[
            (contentBasedRecommendedRecipes['Rating'] >= m) & (contentBasedRecommendedRecipes['Rating'].notnull()) & (
                contentBasedRecommendedRecipes['AggregatedRating'].notnull())]
        qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
        qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
        qualifiedRecipes2['wr'] = qualifiedRecipes2.apply(weightedRating, axis=1)
        qualifiedRecipes2 = qualifiedRecipes2.sort_values('wr', ascending=False).head(12)
        return qualifiedRecipes2.to_json()

@app.route('/collaborative', methods=['POST'])
def getCollaborativeRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return getCollaborativeRecommendations(likedRecipeList)  
    return pd.DataFrame()

def getCollaborativeRecommendations(recipe_ids):
    recommendItemLength = 4
    
    for recipe_id in recipe_ids:
        if recipe_id not in collaborativeMatrix.columns:
            # remove recipe that doesnt have more than {threshold} count of reviews from participating in correlation matching
            recipe_ids.remove(recipe_id)

    recommendRecipeIndices = []
    for recipe_id in recipe_ids:
        users_rating = collaborativeMatrix[recipe_id]
        similar_recipes = collaborativeMatrix.corrwith(users_rating)
        similar_recipes = pd.DataFrame(similar_recipes,columns=['correlation'])
        result_recipes = similar_recipes.join(df_rating['score']).sort_values(by='correlation', ascending=False)
        result_recipes = result_recipes[result_recipes['score'] > 4].sort_values(by = 'correlation', ascending = False) 
        recommendRecipeIndices.extend(result_recipes.iloc[1:recommendItemLength + 1].index.tolist())

    filtered_rows = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId']))
    
    for k in range(len(recommendRecipeIndices)):
        for i in range(len(qualifiedRecipes)):
            if qualifiedRecipes.iloc[i]['RecipeId'] == recommendRecipeIndices[k]:
                filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
                filtered_rows.loc[filtered_rows.index[-1], 'SourceRecipeId'] = recipe_ids[math.floor(k / recommendItemLength)]
                break
    
    returnValue = filtered_rows[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId']]
    return returnValue.to_json()

#print(getCollaborativeRecommendations([56, 10744]))

@app.route('/knn', methods=['POST'])
def getKNNRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return knn_recommender(likedRecipeList)  
    return pd.DataFrame()

def knn_recommender(recipe_ids, matrix = knnMatrix, n_recs = 10, n_result = 4):
    cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
    cf_knn_model.fit(matrix)
    cf_recs = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId']))
    
    for recipe_id in recipe_ids:
        recipe_index = process.extractOne(str(recipe_id),qualifiedRecipes['RecipeId'])[2]
        distances, indices = cf_knn_model.kneighbors(matrix.iloc[recipe_index,:].values.reshape(1,-1), n_neighbors=n_recs)
        recipe_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        iterNo = range(len(recipe_rec_ids))
        
        for i in iterNo:
            if i == n_result:
                break
            index_to_append = recipe_rec_ids[i][0]
            if not any(cf_recs['RecipeId'] == qualifiedRecipes.iloc[index_to_append]['RecipeId']):
                cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
                cf_recs.loc[cf_recs.index[-1], 'SourceRecipeId']  = recipe_id
            else:
                iterNo = iterNo + 1
    
    cf_recs = cf_recs[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId']]
    
    return cf_recs

#print(knn_recommender([56, 26499]))

@app.route('/datasetDump')
def getDatasetDump():
    return datasetDumpAlgolia.to_json()


if __name__ == "__main__":
    osPort = os.getenv("PORT")
    if osPort == None:
        port = 5000
    else:
        port = int(osPort)
    app.run(host='0.0.0.0', port=port)

# class ModelDataHolder:
#     def __init__(self, matrix)

C:\Users\CY\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


['AggregatedRating', 'Images', 'Name', 'Rating', 'RecipeCategory', 'RecipeId', 'ReviewCount', 'T', 'TotalTime', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__dataframe__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__r

C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from p

  RecipeId                                              Name  \
0   181195                                  Cilantro Chicken   
1    77854                     Kittencal's Best Deviled Eggs   
2   120835               Kittencal's Easy Creamy White Glaze   
3    70239  Cracker Barrel's Hash Browns Casserole - Copycat   
4    41156             Strawberry Shortcake a la Treebeard's   
5    65768                                    Chicken Lazone   
6    11266                     Master Brine Recipe for Meats   
7    13745                                     Jello Cookies   

                                              Images TotalTime  \
0  c("https://img.sndimg.com/food/image/upload/w_...     PT55M   
1  c("https://img.sndimg.com/food/image/upload/w_...     PT25M   
2  c("https://img.sndimg.com/food/image/upload/w_...      PT5M   
3  c("https://img.sndimg.com/food/image/upload/w_...   PT1H10M   
4  c("https://img.sndimg.com/food/image/upload/w_...   PT1H20M   
5  c("https://img.sndimg.co

C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
C:\Users\CY\AppData\Local\Temp\ipykernel_19608\3688392536.py:233: FutureWarning: The frame.append method is deprecated and will be removed from p

  RecipeId                                               Name  \
0    50004                                        Lil Smokies   
1   113983                               Classic Saffron Rice   
2    10958                        Cheating Scalloped Potatoes   
3   167914  The Best Fluffiest Buttermilk Pancakes on the ...   
4    63689      My Family's Favorite Sloppy Joes (Pizza Joes)   
5    69173   Kittencal's Italian Melt-In-Your-Mouth Meatballs   
6    28943                  Most Incredible No Fail Pie Crust   
7    32844            Pasta with Sausage, Tomatoes, and Cream   

                                              Images TotalTime RecipeCategory  \
0  c("https://img.sndimg.com/food/image/upload/w_...     PT35M   Lunch/Snacks   
1  c("https://img.sndimg.com/food/image/upload/w_...     PT22M     White Rice   
2  c("https://img.sndimg.com/food/image/upload/w_...   PT1H40M   Lunch/Snacks   
3  c("https://img.sndimg.com/food/image/upload/w_...     PT10M      Breakfast   
4  c("htt